<a href="https://colab.research.google.com/github/Super-rookie-Py/_WebProgramming_Class/blob/master/%EB%B0%95%EA%B1%B4%EC%9A%B0__%EA%B5%AC%EA%B8%80_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [33]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1,428 kB]
Fet

## 필요 라이브러리 import

In [34]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

## `scroll_down()`: 스크롤을 내리는 함수

In [35]:
def scroll_down():
  scroll_count = 0
  print("[scroll_down(): 스크롤 다운 시작]")

  last_height = wd.execute_script("return document.body.scrollHeight")
  after_click = False

  while True:

    print(f"[스크롤 다운: {scroll_count}]")
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_count += 1
    time.sleep(1)

    new_height = wd.execute_script("return document.body.scrollHeight")

    if last_height == new_height:
      if after_click is True:
        break
      else:
        try: 
          more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e)
          break
    last_height = new_height

## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [48]:
def click_and_save(dir_name, index, img, img_list_length):  
  global scraped_count

  try:
    img.click()
    wd.implicitly_wait(3)
    src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')
    if src.split('.')[-1] == 'png':
      urlretrieve(src, dir_name + '/' +str(scraped_count+1)+'.png')
      print(f"{index+1}/{img_list_length} PNG 이미지 저장")
    else:
      urlretrieve(src, dir_name + '/' +str(scraped_count+1)+'.jpg')
      print(f"{index+1}/{img_list_length} JPG 이미지 저장")
  
    scraped_count += 1

  except HTTPError as e:
    print(e)
    pass
  except ElementClickInterceptedException as e:
    print(e)
    pass

## `scraping()`: 구글 이미지 스크래핑 시작

In [37]:
def scraping(dir_name, query):
  global scraped_count

  url = f"https://www.google.com/search?q={query}&tbm=isch&tbs=isz%3Al&hl=ko&sa=X&ved=0CAEQpwVqFwoTCPjR0aXM6-sCFQAAAAAdAAAAABAC&biw=1287&bih=665"
  wd.get(url)
  wd.maximize_window()

  scroll_down()

  div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
  img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')

  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list))
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY+100")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except NoSuchElementException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY+100")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionResetError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e:
      print(e)
      pass
    except socket.gaierror as e:
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      break
  try:      
    print("[스크래핑 종료 (성공률: %.2f%%)]" % (scraped_count/ len(img_list)*100))
  except ZeroDivisionError as e:
    print(e)

  wd.quit()

## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [38]:
def filter_and_remove(dir_name, query, filter_size):
  filtered_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)):
    try:
      file_path = os.path.join(dir_name, file_name)
      img = Image.open(file_path)

      if img.width < filter_size and img.height< filter_size:
        img.close()
        os.remove(file_path)
        print(f"{index} 이미지 제거")
        filtered_count += 1
    except OSError as e:
      print(e)
      os.remove(file_path)
      filtered_count += 1

  print(f"[이미지 제거 개수 : {filtered_count}/{scraped_count}]")

## 이미지 확인

In [50]:
socket.setdefaulttimeout(30)

wd = webdriver.Chrome('chromedriver', options=chrome_options)
scraped_count = 0

path ='./'
query = input("검색어 입력: ")

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
filter_and_remove(dir_name, query, 400)

검색어 입력: forest
[./forest 디렉토리 생성]
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]
[스크롤 다운: 10]
[스크롤 다운: 11]
1/849 JPG 이미지 저장
2/849 JPG 이미지 저장
3/849 JPG 이미지 저장
4/849 JPG 이미지 저장
5/849 JPG 이미지 저장
6/849 JPG 이미지 저장
HTTP Error 403: Forbidden
8/849 JPG 이미지 저장
HTTP Error 403: Forbidden
10/849 JPG 이미지 저장
11/849 JPG 이미지 저장
12/849 JPG 이미지 저장
13/849 JPG 이미지 저장
14/849 JPG 이미지 저장
15/849 JPG 이미지 저장
16/849 JPG 이미지 저장
17/849 JPG 이미지 저장
18/849 JPG 이미지 저장
19/849 JPG 이미지 저장
20/849 JPG 이미지 저장
21/849 JPG 이미지 저장
22/849 JPG 이미지 저장
23/849 JPG 이미지 저장
24/849 JPG 이미지 저장
25/849 JPG 이미지 저장
26/849 JPG 이미지 저장
27/849 JPG 이미지 저장
28/849 JPG 이미지 저장
29/849 JPG 이미지 저장
30/849 JPG 이미지 저장
31/849 JPG 이미지 저장
32/849 JPG 이미지 저장
33/849 JPG 이미지 저장
34/849 JPG 이미지 저장
35/849 JPG 이미지 저장
36/849 JPG 이미지 저장
37/849 JPG 이미지 저장
38/849 JPG 이미지 저장
39/849 JPG 이미지 저장
40/849 JPG 이미지 저장
41/849 JPG 이미지 저장
42/849 JPG 이미지 저장
43/849 JPG 이미지 저장
44/849 JPG

In [52]:
!ls forest

106.jpg  174.jpg  271.jpg  352.jpg  441.jpg  510.jpg  596.jpg  66.jpg	753.jpg
107.jpg  175.jpg  272.jpg  354.jpg  444.jpg  511.jpg  597.jpg  674.jpg	754.jpg
110.jpg  178.jpg  274.jpg  356.jpg  446.jpg  512.jpg  598.jpg  675.jpg	755.jpg
111.jpg  179.jpg  276.jpg  357.jpg  447.jpg  514.jpg  5.jpg    678.jpg	758.jpg
112.jpg  185.jpg  278.jpg  358.jpg  448.jpg  515.jpg  600.jpg  680.jpg	760.jpg
114.jpg  186.jpg  279.jpg  359.jpg  449.jpg  516.jpg  602.jpg  681.jpg	761.jpg
115.jpg  188.jpg  27.jpg   360.jpg  44.jpg   51.jpg   606.jpg  685.jpg	763.jpg
116.jpg  18.jpg   281.jpg  364.jpg  450.jpg  521.jpg  607.jpg  68.jpg	764.jpg
121.jpg  190.jpg  283.png  366.jpg  451.jpg  532.jpg  608.jpg  690.jpg	767.jpg
122.jpg  191.jpg  284.jpg  367.jpg  452.jpg  533.jpg  60.jpg   695.jpg	770.jpg
123.jpg  193.jpg  289.jpg  369.jpg  453.jpg  543.jpg  611.jpg  696.jpg	772.jpg
125.jpg  195.jpg  290.jpg  371.jpg  457.jpg  544.jpg  612.jpg  697.jpg	773.jpg
126.jpg  19.jpg   292.jpg  372.jpg  458.jpg  549.jpg  

In [54]:
from google.colab import files
files.download('./forest/13.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>